In [1]:
from os import listdir
import re
import pandas as pd
import numpy as np

#Pickle
from pickle import dump
from pickle import load

from keras.models import load_model

#Keras
#Tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences

#NTLK
import nltk
#Lemmatization
from nltk.stem import WordNetLemmatizer
#corpus-bleu evaluation metrics
from nltk.translate.bleu_score import corpus_bleu
from tensorflow.keras.layers import Input
from tqdm import tqdm

from PIL import Image
from IPython.display import Image,display

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Num GPUs Available:  1
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10387594718386175524
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2254123828
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16629678345834780323
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [3]:
TRAIN_TOKENIZER_PATH = "D:/SiDi/Project/Modulo III/dataset/train_tokenizer.pkl"

In [4]:
max_length = 64
def word_for_id(integer , tokenizer):
    for word,index in tokenizer.word_index.items():
        if index == integer:
            return word
        
def generate_desc(model , tokenizer , photo , max_length):
    
    input_text = 'startseq'
    
    for i in range(max_length):
        
        sequence = tokenizer.texts_to_sequences([input_text])[0]
        sequence = pad_sequences([sequence] , maxlen=max_length)
        # predict the next word
        next_word_id = model.predict([photo,sequence],verbose = 0)
        
        # get highest probality word from list of words
        next_word_id = np.argmax(next_word_id)
        
        # get word from id
        word = word_for_id(next_word_id , tokenizer)
        
        if word is None:
            break
            
        # update input text
        input_text += ' '+ word
        
        if word == 'endseq':
            break
            
    return input_text

In [5]:
def extract_features_for_one_image(filename):
    """model = VGG16()
    model.layers.pop()
    
    model = Model(inputs = model.inputs , outputs = model.layers[-1].output)
    
    #image = load_img(filename , target_size=(224,224))"""
    model = VGG16()   
    #model = VGG16( include_top=False,input_tensor=Input(shape=(224, 224, 3)))

    #remove last layer
    model = Model(inputs = model.inputs , outputs = model.layers[-2].output)

    image = load_img(filename , target_size=(224,224))

    image = img_to_array(image)
    
    image = image.reshape((1,image.shape[0],image.shape[1],image.shape[2]))
    
    image = preprocess_input(image)
    
    feature = model.predict(image , verbose = 0)
    
    return feature

In [6]:

import tensorflow as tf
def get_image_caption(filename,model):
    tokenizer = load(open(TRAIN_TOKENIZER_PATH,'rb'))
    photo = extract_features_for_one_image(filename)
    gpus = tf.config.list_logical_devices('GPU')

    type(gpus)
    if gpus:
      # Replicate your computation on multiple GPUs
      c = []
      for gpu in gpus:
        with tf.device(gpu.name):
            print(gpu.name)
            pass
    gpu = gpus[0]
    
    with tf.device(gpu.name):
        desc = generate_desc(model , tokenizer , photo , 64)
        desc = desc.replace('startseq','')
        desc = desc.replace('endseq','')
    return desc.title()

In [7]:
model2 =  load_model('./model/final_model.h5')

In [9]:
filename = './test/000000363784.jpg'
print(get_image_caption(filename,model2))
#display(Image(filename))

/device:GPU:0
 Um Gato Está Sentado Em Uma Cadeira De Madeira 
